In [1]:
import pandas as pd
import re

df = pd.read_csv('raw_data.csv', delimiter=';')

In [2]:
df

,Name,Full Price,Current Price,Quantity,Category,Subcategory
0,Giovanni Rana Gnocchetti freschi di patate,"2,40","1,11",500 g,Pasta e Riso,Gnocchi
1,GNOCCHI DI PATATE CON ZUCCA,"1,92","0,48",400 g,Pasta e Riso,Gnocchi
2,Gnocchi di Patate,"2,79","1,19",500 g,Pasta e Riso,Gnocchi
3,Gnocchetti Di Patate Freschi,"1,00","0,89",500 g,Pasta e Riso,Gnocchi
4,GNOCCHI CASALINGHI,"1,99","0,37",500 g,Pasta e Riso,Gnocchi
...,...,...,...,...,...,...
8403,Ultima Cat Appetito Difficile Trota,NaN,"2,04",340 g,Amici Animali,Snack per gatti
8404,"Ultima, gatto Adult sterilizzato alimento con ...",NaN,"7,59","1,5 kg",Amici Animali,Snack per gatti
8405,"Purina, gatto Felix Le Ghiottonerie le varietà...",NaN,"5,08",1000 g,Amici Animali,Snack per gatti
8406,"Sheba - Pate' Classic, Con Salmone",NaN,"0,62",85 g,Amici Animali,Snack per gatti


Turning all values to lowercase. Useful to standardize the data and reduce the possibility of errors due to different text formatting.

In [3]:
df['Quantity'] = df['Quantity'].str.lower()
df['Name'] = df['Name'].str.lower()
df['Category'] = df['Category'].str.lower()
df['Subcategory'] = df['Subcategory'].str.lower()

Creating a new column to separate units of measure and the respective value.

In [4]:
df['Units of Measurement'] = df['Quantity'].str.split(' ', n=1).str[1]
df['Quantity'] = df['Quantity'].str.split().str[0]

Convert the values in the 'Full Price' and 'Current Price' columns from strings with the decimal separator ',' to floating-point (float) numbers using the dot as the decimal separator.

In [5]:
df['Full Price'] = df['Full Price'].str.replace(',', '.').astype(float)
df['Current Price'] = df['Current Price'].str.replace(',', '.').astype(float)

Convert 'Quantity' values to int where possible, otherwise will return Na to indicate that the data is missing.

In [6]:
def string_to_int(row):
    try:
        return int(row)
    except ValueError:
        return pd.NA

df['Quantity'] = df['Quantity'].map(string_to_int)

For better understanding of the data, the 'Full price' column will contain the full price of the product, as it now contains a value only if the product is discounted. The 'Current Price' column will contain the discounted product value where present, plus a 'Discount (%)' column will be created which will contract the discount percentage.

In [7]:
# If full price NaN -> full price = current price
df.loc[pd.isnull(df['Full Price']), 'Full Price'] = df['Current Price']

# If current price = full price -> no discount -> Current price = NaN
df.loc[df['Current Price'] == df['Full Price'], 'Current Price'] = pd.NA

# Creating discount percentage column where the discount is present
def percentage(row):
    current_price = row['Current Price']
    full_price = row['Full Price']
    if current_price != 'NaN':
        discount = ((full_price - current_price) / full_price) * 100
        return round(discount, 2)
    else:
        return pd.NA
    
df['Discount (%)'] = df.apply(lambda row : percentage(row), axis=1)

# Renaming columns
df.rename(columns = {'Full Price':'Price', 'Current Price': 'Discounted Price'}, inplace = True)

# Reordering columns
df = df[['Name', 'Price', 'Discounted Price', 'Discount (%)', 'Quantity', 'Units of Measurement', 'Category', 'Subcategory']]

# Dropping duplicates
df = df.drop_duplicates()

Because the data frame contains ambiguous values in the 'Units of Measurement' column such as the presence of inappropriate digits and values. I created a list of appropriate units of measurement for data cleaning purposes. Using this list, I removed all rows containing units of measurement that were not present in it. By conducting this data cleaning process, I simplified my dataset and made it more comprehensible, while also eliminating potential errors or inconsistencies in the measured values. 

In [8]:
unique_units_list = []
ok_units = ['gr', 'g', 'ml', 'lt', 'kg', 'pz']

for unit in df['Units of Measurement'].unique():
    if unit not in ok_units:
        unique_units_list.append(unit)

df_units_check = df[df['Units of Measurement'].isin(unique_units_list)]

df = df[df['Units of Measurement'].isin(ok_units)]

df['Units of Measurement'] = df['Units of Measurement'].replace('g', 'gr')

The next step of cleaning the data frame involves the elimination of all products whose names are incomplete, specifically those that end with '...'. This is necessary to ensure data accuracy and reliability in the dataset.
By removing products with incomplete names, we can avoid any potential confusion or misinterpretation during data analysis.

In [9]:
regex = r'.+\.\.\.$'

def match_regex(product_name):
    return re.match(regex, product_name) is not None

incomplete_names = df[df['Name'].apply(match_regex)]['Name'].tolist()

df = df[~df['Name'].isin(incomplete_names)]

In [10]:
df.reset_index(drop=True, inplace=True)
df

,Name,Price,Discounted Price,Discount (%),Quantity,Units of Measurement,Category,Subcategory
0,giovanni rana gnocchetti freschi di patate,2.40,1.11,53.75,500,gr,pasta e riso,gnocchi
1,gnocchi di patate con zucca,1.92,0.48,75.00,400,gr,pasta e riso,gnocchi
2,gnocchi di patate,2.79,1.19,57.35,500,gr,pasta e riso,gnocchi
3,gnocchetti di patate freschi,1.00,0.89,11.00,500,gr,pasta e riso,gnocchi
4,gnocchi casalinghi,1.99,0.37,81.41,500,gr,pasta e riso,gnocchi
...,...,...,...,...,...,...,...,...
6167,whiskas anti-hairball,1.74,NaN,NaN,60,gr,amici animali,snack per gatti
6168,whiskas pranzetti in salsa 1+ anni,2.66,NaN,NaN,300,gr,amici animali,snack per gatti
6169,ultima cat appetito difficile trota,2.04,NaN,NaN,340,gr,amici animali,snack per gatti
6170,"sheba - pate' classic, con salmone",0.62,NaN,NaN,85,gr,amici animali,snack per gatti
